<a href="https://colab.research.google.com/github/mingyung-park/NLP_Code/blob/main/Colab_Tutorial_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/NLP/experiments/
# 깃허브의 install_packages.sh install 하기
!chmod +x install_packages.sh
!./install_packages.sh

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/NLP/experiments


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP/experiments/module

from google.colab import files
# uploaded = files.upload()  # .env 파일을 업로드. 로컬에 저장되어있으면 안해도 됨
from dotenv import load_dotenv
import os

with open('.env', 'w') as f: #.env 파일에 API KEY 설정해야 돌아갑니다!! 이 코드는 파일에 직접 apikey 저장
    f.write('UPSTAGE_API_KEY=up_c68imJzmRpRUGw7Ww77fwFhpmLxWu\n')

/content/drive/MyDrive/Colab Notebooks/NLP/experiments/module


In [ ]:
from docs_loaders import DocumentLoader
from text_splitters import TextSplitter
from vector_stores import VectorStore
from retrievers import Retriever
from prompts import Prompts
from experiment_runner import ExperimentLoader
import utils
import pandas as pd
import json
import os

In [ ]:
# path 설정
module_path = './module'
test_data_path = './data'
prompt_folder_path = './module/templates/'
local_db_path = '../faiss_db/'

ex_option_path = './ex_options.json'
prompt_option_path = './prompt_options.json'
kb_path='../database/ewha.pdf'

In [ ]:
# test data load
test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP/experiments/testset.csv")
test_df

,prompts,answers
0,QUESTION1) 재학 중인 학생이 휴학을 하려면 학기 개시일로부터 며칠 이내에 ...,(D)
1,QUESTION2) '재입학은 a회에 한하여 할 수 있다. 다만 제 28조제4호에 ...,(A)
2,QUESTION3) 학생이 소속 학과 또는 전공 이외의 전공 교과목을 총장이 정하는...,(C)
3,QUESTION4) 다음 보기의 학생들 중 제적을 당하지 않는 사람을 고르면?\n(...,(D)
4,QUESTION5) 2019학년도 휴먼기계바이오공학부의 입학 정원은 몇 명인가? \...,(C)
5,QUESTION6) 1980학년도 이전 입학생에 대하여 적용하는 등급에 따른 성적점...,(B)
6,QUESTION7) 사회체육학과 소속 학생에게 수여하는 학위는 무엇인가? \n(A)...,(D)
7,QUESTION8) 복수전공 신청 자격에 해당하지 않는 것은? \n(A) 1학년을 ...,(C)
8,QUESTION9) 이화여자대학교의 설립 정신은 무엇인가요? (A) 공산주의 이념...,(C)
9,QUESTION10) 이화여자대학교의 위치는 어디인가요? (A) 강남구 (B) ...,(B)


In [ ]:
experiment_loader = ExperimentLoader(
    kb_path='../database/ewha.pdf',
    config_path='./ex_options.json',
    prompt_options_path="./prompt_options.json",
    prompt_folder_path="./templates/"
    )
#df = experiment_loader.execute_scenario("scenario1",test_df)
#df["default"] # default 프롬프트에 대한 결과

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP/experiments/module
import utils
# 시나리오 1 실행
df_scenario1 = experiment_loader.execute_scenario("scenario1", test_df)
df_scenario1["default"]["pred_answer"] = df_scenario1["default"]["Response"].apply(utils.extract_answer)
df_scenario1["few_shot_prompt"]["pred_answer"] = df_scenario1["few_shot_prompt"]["Response"].apply(utils.extract_answer)
acc_scenario1=utils.calculate_accuracy(df_scenario1["default"])
acc_scenario1_few=utils.calculate_accuracy(df_scenario1["few_shot_prompt"])
df_scenario1["default"].to_csv("../results/scenario_1_default_results.csv", index=False)  # CSV로 저장
df_scenario1["few_shot_prompt"].to_csv("../results/scenario_1_fewshot_results.csv", index=False)  # CSV로 저장

/content/drive/MyDrive/Colab Notebooks/NLP/experiments/module


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{0: 'f168ceb4-93ff-4cc5-8870-8dd3cca6852e', 1: '104ea008-4ebf-4da8-a266-94697522085d', 2: '05742abd-3373-43ba-aab9-7c01eed60604', 3: 'aa7bb725-1d57-42ef-a871-360d15c9e7d2', 4: '0181bda4-7317-44a1-afb7-4c0ed9e54786', 5: '3408528e-32c1-4591-aac9-1b40b476382d', 6: 'b71ad9db-175f-4276-946b-5bd2354f0505', 7: '433b64e1-8310-4633-8179-9aea22f9a309', 8: '2fb8b04e-8421-493a-8a0d-2ec9bd954fa3', 9: '209ed42d-8a20-4973-ae39-5d30fe3067d4', 10: 'aff6d413-cd69-4964-b215-207197959c63', 11: 'cad20555-ce5c-4aac-ac6c-370cde7c8e04', 12: '3e9e29e3-fd4b-4578-8fe8-2762eeeb57ce', 13: 'e174101c-9712-4314-8b21-f839be775f79', 14: '06cd4767-a094-495e-8629-1bc4d8c523e2', 15: '731ca944-9b0b-4e8f-9361-cea37efd00d9', 16: 'c608f7d9-6361-4048-8623-5b055adcbd11', 17: 'bd538b18-d756-4fe7-b317-fa99d18873dd', 18: '70a5be8c-468c-43c0-b93d-cd19650c63db', 19: '959da439-94e0-4bb6-96dc-e8b0ebed0388', 20: '4585418b-9834-4a6f-b88c-079e350aa815', 21: '18357fe4-8427-422b-a0cc-7365c19b8c2c', 22: 'c41aa8b7-39ae-4aa8-b2a7-a275231a3f68

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP/experiments/module
import utils
# 시나리오 2 실행
df_scenario2 = experiment_loader.execute_scenario("scenario2", test_df)
df_scenario2["default"]["pred_answer"] = df_scenario2["default"]["Response"].apply(utils.extract_answer)
df_scenario2["few_shot_prompt"]["pred_answer"] = df_scenario2["few_shot_prompt"]["Response"].apply(utils.extract_answer)
acc_scenario2=utils.calculate_accuracy(df_scenario2["default"])
acc_scenario2_few=utils.calculate_accuracy(df_scenario2["few_shot_prompt"])
df_scenario2["default"].to_csv("../results/scenario_2_default_results.csv", index=False)  # CSV로 저장
df_scenario2["few_shot_prompt"].to_csv("../results/scenario_2_fewshot_results.csv", index=False)  # CSV로 저장


/content/drive/MyDrive/Colab Notebooks/NLP/experiments/module


In [ ]:
# # 시나리오 3 실행
%cd /content/drive/MyDrive/Colab Notebooks/NLP/experiments/module
import utils
df_scenario3 = experiment_loader.execute_scenario("scenario3", test_df)
df_scenario3["default"]["pred_answer"] = df_scenario3["default"]["Response"].apply(utils.extract_answer)
df_scenario3["few_shot_prompt"]["pred_answer"] = df_scenario3["few_shot_prompt"]["Response"].apply(utils.extract_answer)
acc_scenario3=utils.calculate_accuracy(df_scenario3["default"])
acc_scenario3_few=utils.calculate_accuracy(df_scenario3["few_shot_prompt"])
df_scenario3["default"].to_csv("../results/scenario3_default_results.csv", index=False)  # CSV로 저장
df_scenario3["few_shot_prompt"].to_csv("../results/scenario3_fewshot_results.csv", index=False)  # CSV로 저장


/content/drive/MyDrive/Colab Notebooks/NLP/experiments/module
{0: '678a325a-2187-468a-a16d-f3c430703db8', 1: '316e30e6-81fb-4350-94b2-879190ffe355', 2: 'e1ab4291-7b49-4f04-8404-fc9ab655d5d3', 3: '622b118f-87d9-4993-aa88-2c6666bf7eae', 4: '8862ceb5-b5cb-4c06-a180-dc6624c01eee', 5: '932df802-531f-430c-aac5-8f5bddc0d5c7', 6: '088b6fa9-7e4c-4be1-a0f1-965d04a21397', 7: 'd0cd8a5d-229c-4335-8f8d-511799896960', 8: '7018e0ed-71a5-40b7-bc6e-c9b7fd1a799c', 9: '8c55405f-b03a-4459-a3f6-f963fb76150d', 10: '2064bfd1-6e96-4dbc-aff5-eabba0fa5503', 11: 'f5ed0dd0-be59-4e76-a8f4-407991efc5a9', 12: '1473310e-565c-4609-b550-5e4f6d3f0bb7', 13: 'c5001294-ead1-4ff8-b11f-76551c4b73af', 14: 'f7b05077-a5d5-4bb9-b2e3-c9b35c611c5b', 15: 'ab627de8-ac18-41f2-a0e2-ac49c405bb3d', 16: 'e068d332-590e-426c-9297-ef5f166cf8ee', 17: '09a08c8e-8c5e-415c-97ac-7f45179cc0c4', 18: 'a27a008b-4b20-47e2-83af-2e0fc9a108a7', 19: 'c0efcb46-3339-4fa6-8d95-f995a7506085', 20: 'cd671399-6f6a-4291-b204-5cf441612eb1', 21: '36bd911d-8310-4a6e

In [ ]:
# 시나리오 4 실행
%cd /content/drive/MyDrive/Colab Notebooks/NLP/experiments/module
import utils
df_scenario4 = experiment_loader.execute_scenario("scenario4", test_df)
df_scenario4["default"]["pred_answer"] = df_scenario4["default"]["Response"].apply(utils.extract_answer)
df_scenario4["few_shot_prompt"]["pred_answer"] = df_scenario4["few_shot_prompt"]["Response"].apply(utils.extract_answer)
acc_scenario4=utils.calculate_accuracy(df_scenario4["default"])
acc_scenario4_few=utils.calculate_accuracy(df_scenario4["few_shot_prompt"])
df_scenario4["default"].to_csv("../results/scenario4_default_results.csv", index=False)  # CSV로 저장
df_scenario4["few_shot_prompt"].to_csv("../results/scenario4_fewshot_results.csv", index=False)  # CSV로 저장
print(acc_scenario4,acc_scenario4_few)

/content/drive/MyDrive/Colab Notebooks/NLP/experiments/module
Using local vector database: ../faiss_db/structurebase
Vector Store loaded from: ../faiss_db/structurebase
setup_retriever...
db retriever
setup_prompt...
prompt_option_name:default
complete invoke default
prompt_option_name:few_shot_prompt
complete invoke few_shot_prompt
54.0 54.0


In [ ]:
print([acc_scenario1,acc_scenario1_few,acc_scenario2,acc_scenario2_few,acc_scenario3,acc_scenario3_few,acc_scenario4,acc_scenario4_few])

In [ ]:
# # 시나리오 5 실행
%cd /content/drive/MyDrive/Colab Notebooks/NLP/experiments/module
import utils
experiment_loader = ExperimentLoader(
    kb_path='../database/ewha.pdf',
    config_path='./ex_options.json',
    prompt_options_path="./prompt_options.json",
    prompt_folder_path="./templates/"
    )
df_scenario5 = experiment_loader.execute_scenario("scenario5", test_df)
df_scenario5["default"]["pred_answer"] = df_scenario5["default"]["Response"].apply(utils.extract_answer)
df_scenario5["few_shot_prompt"]["pred_answer"] = df_scenario5["few_shot_prompt"]["Response"].apply(utils.extract_answer)
acc_scenario5=utils.calculate_accuracy(df_scenario5["default"])
acc_scenario5_few=utils.calculate_accuracy(df_scenario5["few_shot_prompt"])
df_scenario5["default"].to_csv("../results/scenario5_default_results.csv", index=False)  # CSV로 저장
df_scenario5["few_shot_prompt"].to_csv("../results/scenario5_fewshot_results.csv", index=False)  # CSV로 저장
acc_scenario5

/content/drive/MyDrive/Colab Notebooks/NLP/experiments/module
hello
[Document(metadata={'category': 'table', 'html': '<table id="119" style="font-size:20px"><tr><td>등급</td><td>성적점</td></tr><tr><td>A+</td><td>4.3</td></tr><tr><td>Ao</td><td>4.0</td></tr><tr><td>A-</td><td>3.7</td></tr><tr><td>B+</td><td>3.3</td></tr><tr><td>Bo</td><td>3.0</td></tr><tr><td>B-</td><td>2.7</td></tr><tr><td>C+</td><td>2.3</td></tr><tr><td>Co</td><td>2.0</td></tr><tr><td>C-</td><td>1.7</td></tr><tr><td>D+</td><td>1.3</td></tr><tr><td>Do</td><td>1.0</td></tr><tr><td>D-</td><td>0.7</td></tr><tr><td>F</td><td>0</td></tr><br/><caption id="118" style="font-size:16px">제39조(성적) ① 학업성적은 시험 또는 평소의 성적을 참작하여 다음과 같이 분류 표시하고<br/>등급에 따라 학점마다 성적점을 주며, D-이상은 급제, F는 낙제로 한다.</caption></table>'}, page_content='등급성적점A+4.3Ao4.0A-3.7B+3.3Bo3.0B-2.7C+2.3Co2.0C-1.7D+1.3Do1.0D-0.7F0제39조(성적) ① 학업성적은 시험 또는 평소의 성적을 참작하여 다음과 같이 분류 표시하고등급에 따라 학점마다 성적점을 주며, D-이상은 급제, F는 낙제로 한다.'), Document(metadata={'category': 'table', 'html': '<table id

66.0

In [ ]:
print(acc_scenario5,acc_scenario5_few)

66.0 57.99999999999999


In [ ]:
df_scenario6 = experiment_loader.execute_scenario("scenario6", test_df)
df_scenario6["default"]["pred_answer"] = df_scenario6["default"]["Response"].apply(utils.extract_answer)
df_scenario6["few_shot_prompt"]["pred_answer"] = df_scenario6["few_shot_prompt"]["Response"].apply(utils.extract_answer)
acc_scenario6=utils.calculate_accuracy(df_scenario6["default"])
acc_scenario6_few=utils.calculate_accuracy(df_scenario6["few_shot_prompt"])
df_scenario6["default"].to_csv("../results/scenario6_default_results.csv", index=False)  # CSV로 저장
df_scenario6["few_shot_prompt"].to_csv("../results/scenario6_fewshot_results.csv", index=False)  # CSV로 저장
acc_scenario6

Using local vector database: ../faiss_db/htmlbase_base
Vector Store loaded from: ../faiss_db/htmlbase_base
setup_retriever...
db retriever
setup_prompt...
prompt_option_name:default
complete invoke default
prompt_option_name:few_shot_prompt
complete invoke few_shot_prompt


57.99999999999999

In [ ]:
print(acc_scenario6,acc_scenario6_few)

57.99999999999999 60.0


---
# 결과

In [ ]:
import pandas as pd
df_1 = pd.read_csv("../results/scenario_1_default_results.csv")
df_1_few=pd.read_csv("../results/scenario_1_fewshot_results.csv")
df_2 = pd.read_csv("../results/scenario_2_default_results.csv")
df_2_few=pd.read_csv("../results/scenario_2_fewshot_results.csv")
df_3 = pd.read_csv("../results/scenario3_default_results.csv")
df_3_few=pd.read_csv("../results/scenario3_fewshot_results.csv")
df_4 = pd.read_csv("../results/scenario4_default_results.csv")
df_4_few=pd.read_csv("../results/scenario4_fewshot_results.csv")
df_5 = pd.read_csv("../results/scenario5_default_results.csv")
df_5_few=pd.read_csv("../results/scenario5_fewshot_results.csv")
df_6 = pd.read_csv("../results/scenario6_default_results.csv")
df_6_few=pd.read_csv("../results/scenario6_fewshot_results.csv")

acc_1=utils.calculate_accuracy(df_1)
acc_1_few=utils.calculate_accuracy(df_1_few)
acc_2=utils.calculate_accuracy(df_2)
acc_2_few=utils.calculate_accuracy(df_2_few)
acc_3=utils.calculate_accuracy(df_3)
acc_3_few=utils.calculate_accuracy(df_3_few)
acc_4=utils.calculate_accuracy(df_4)
acc_4_few=utils.calculate_accuracy(df_4_few)
acc_5=utils.calculate_accuracy(df_5)
acc_5_few=utils.calculate_accuracy(df_5_few)
acc_6=utils.calculate_accuracy(df_6)
acc_6_few=utils.calculate_accuracy(df_6_few)

In [ ]:
print([acc_1,acc_1_few])
print([acc_2,acc_2_few])
print([acc_3,acc_3_few])
print([acc_4,acc_4_few])
print([acc_5,acc_5_few])
print([acc_6,acc_6_few])

[62.0, 56.00000000000001]
[56.00000000000001, 57.99999999999999]
[57.99999999999999, 54.0]
[54.0, 54.0]
[66.0, 57.99999999999999]
[57.99999999999999, 60.0]
